[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/python/single_point.ipynb)

# Single Point

`janus-core` contains various machine learnt interatomic potentials (MLIPs), including MACE based models (MACE-MP, MACE-OFF), CHGNet, Orb and more, full list on https://github.com/stfc/janus-core.

Other will be added as their utility is proven beyond a specific material.

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,orb,chgnet,visualise] data-tutorials --system # Install janus-core with MACE, Orb, CHGNet, and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Prepare data

Use `data_tutorials` to get the data required for this tutorial:

In [ ]:
from data_tutorials.data import get_data

get_data(
    url="https://raw.githubusercontent.com/stfc/janus-core/main/docs/source/tutorials/data/",
    filename=["sucrose.xyz", "NaCl-set.xyz"],
    folder="data",
)

## Single point calculations for periodic system

In [ ]:
from ase.build import bulk

from janus_core.calculations.single_point import SinglePoint

# Change to cuda if you have a gpu or mps for apple silicon
device = "cpu"

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)
sp = SinglePoint(
    struct=NaCl,
    arch="mace_mp",
    device=device,
    model="small",
    calc_kwargs={"default_dtype": "float64"},
)
res_mace = sp.run()

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)
sp = SinglePoint(
    struct=NaCl,
    arch="orb",
    device=device,
)
res_orb = sp.run()

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)
sp = SinglePoint(
    struct=NaCl,
    arch="chgnet",
    device=device,
)
res_chgnet = sp.run()

print(f"    MACE[eV]: {res_mace['energy']}")
print(f"Orb[eV]: {res_orb['energy']}")
print(f"  CHGNeT[eV]: {res_chgnet['energy']}")

## Simple Molecules

In [ ]:
from ase.build import molecule
from weas_widget import WeasWidget

from janus_core.calculations.single_point import SinglePoint

sp = SinglePoint(
    struct=molecule("H2O"),
    arch="mace_off",
    device=device,
    model="medium",
)
res = sp.run()
print(res)
v=WeasWidget()
v.from_ase(sp.struct)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v


## Sugar on salt

In [ ]:
from ase.build import add_adsorbate, bulk
from ase.io import read, write

a = 5.63
NaCl = bulk(
    "NaCl", crystalstructure="rocksalt", cubic=True, orthorhombic=True, a=5.63
) * (6, 6, 3)

NaCl.center(vacuum=20.0, axis=2)
sugar = read("data/sucrose.xyz")
add_adsorbate(slab=NaCl, adsorbate=sugar, height=4.5, position=(10, 10))
write("slab.xyz", NaCl)

sp = SinglePoint(
    struct="slab.xyz",
    arch="mace_mp",
    device=device,
    model="small",
)
res = sp.run()
print(res)

v = WeasWidget()
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v.from_ase(NaCl)
v

## Calculate an entire collection of data frames

In [ ]:
from ase.io import read
import matplotlib.pyplot as plt
import numpy as np

frames = read("data/NaCl-set.xyz", format="extxyz", index=":")
dft_energy = np.array([s.info["dft_energy"] / len(s) for s in frames])

sp = SinglePoint(
    struct="data/NaCl-set.xyz",
    arch="mace_mp",
    device=device,
    model="medium-0b3",
)
sp.run()

mace_mp_energy = np.array([s.info["mace_mp_energy"] / len(s) for s in sp.struct])
rmse_mace = np.linalg.norm(mace_mp_energy - dft_energy) / np.sqrt(len(dft_energy))

sp = SinglePoint(struct="data/NaCl-set.xyz", arch="chgnet", device=device)
sp.run()

chgnet_energy = np.array([s.info["chgnet_energy"] / len(s) for s in sp.struct])
rmse_chgnet = np.linalg.norm(chgnet_energy - dft_energy) / np.sqrt(len(dft_energy))

sp = SinglePoint(struct="data/NaCl-set.xyz", arch="orb", device=device)
sp.run()

orb_energy = np.array([s.info["orb_energy"] / len(s) for s in sp.struct])
rmse_orb = np.linalg.norm(orb_energy - dft_energy) / np.sqrt(len(dft_energy))

print(
    f"rmse: mace_mp = {rmse_mace}, chgnet = {rmse_chgnet}, "
    f"orb = {rmse_orb} eV/atom"
)

fig, ax = plt.subplots()

ax.scatter(dft_energy, mace_mp_energy, marker="o", label="mace-mp-0")
ax.scatter(dft_energy, orb_energy, marker="x", label="orb")
ax.scatter(dft_energy, chgnet_energy, marker="+", label="chgnet")
ax.legend()
plt.xlabel("MLIP [eV/atom]")
plt.ylabel("DFT [eV/atom]")

plt.show()